In [9]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy.interpolate import interp1d

# Data for m30s
average_powers_m30s = np.array([
    1154.5, 1357.8, 1584.5, 1833.0, 2089.4,
    2375.1, 2665.2, 2963.0, 3303.1, 3643.4,
])
average_efficiencies_m30s = 1 / np.array([
    34.17705031, 33.9442993, 34.00210623, 34.27667367, 34.80261383,
    35.29884863, 35.92702256, 36.66446198, 37.76226107, 38.43015302,
])

# Data for m30s+
average_powers_m30s_plus = np.array([
    1076.9, 1327.2, 1507.3, 1749.3, 2016.7,
    2312.2, 2612.1, 2921.4, 3263.8, 3583.6,
])
average_efficiencies_m30s_plus = 1 / np.array([
    29.55603251, 29.35333121, 29.36666004, 29.70056472, 30.19869573,
    30.93710236, 31.733587, 32.7793932, 33.69444195, 34.33965532,
])

# Create interpolators
efficiency_interpolator_m30s = interp1d(average_powers_m30s, average_efficiencies_m30s, fill_value="extrapolate")
efficiency_interpolator_m30s_plus = interp1d(average_powers_m30s_plus, average_efficiencies_m30s_plus, fill_value="extrapolate")

def btc_mined(ths_rt, k):
    return k * ths_rt

def ths_rt_given_power(power, miner_type):
    if miner_type == "m30s":
        efficiency = efficiency_interpolator_m30s(power)
    else:
        efficiency = efficiency_interpolator_m30s_plus(power)
    return power * efficiency

def net_profit(power, btc_price, electricity_rate, k, miner_type, duration=1):
    ths_rt = ths_rt_given_power(power, miner_type)
    revenue = btc_mined(ths_rt, k) * btc_price
    electricity_cost = (power / 1000) * electricity_rate * duration
    return revenue - electricity_cost

def plot_net_profit(btc_price, electricity_rate):
    k = 0.0000000914  # Fixed k value
    powers = np.arange(1000, 3601, 50)
    
    # Calculate profits for m30s and m30s+
    profits_m30s = [net_profit(p, btc_price, electricity_rate, k, "m30s") for p in powers]
    profits_m30s_plus = [net_profit(p, btc_price, electricity_rate, k, "m30s+") for p in powers]
    
    # Plotting
    plt.figure(figsize=(12, 8))
    plt.plot(powers, profits_m30s, label='Net Profit (m30s)', color='blue')
    plt.plot(powers, profits_m30s_plus, label='Net Profit (m30s+)', color='green')
    
    max_profit_m30s = max(profits_m30s)
    max_profit_m30s_plus = max(profits_m30s_plus)
    
    # Calculate percentage for the optimal power
    def calculate_percentage(optimal_power, min_power, max_power):
        return ((optimal_power - min_power) / (max_power - min_power)) * 100

    # Check for no profitable power level
    if max_profit_m30s <= 0:
        plt.axhline(0, color='red', linestyle='--', label='No profitable power level (m30s)')
    else:
        optimal_power_m30s = powers[np.argmax(profits_m30s)]
        percent_m30s = calculate_percentage(optimal_power_m30s, min(average_powers_m30s), max(average_powers_m30s))
        plt.axvline(optimal_power_m30s, color='red', linestyle='--', label=f'Optimal Power (m30s): {optimal_power_m30s}W ({percent_m30s:.1f}%)')
    
    if max_profit_m30s_plus <= 0:
        plt.axhline(0, color='purple', linestyle='--', label='No profitable power level (m30s+)')
    else:
        optimal_power_m30s_plus = powers[np.argmax(profits_m30s_plus)]
        percent_m30s_plus = calculate_percentage(optimal_power_m30s_plus, min(average_powers_m30s_plus), max(average_powers_m30s_plus))
        plt.axvline(optimal_power_m30s_plus, color='purple', linestyle='--', label=f'Optimal Power (m30s+): {optimal_power_m30s_plus}W ({percent_m30s_plus:.1f}%)')
    
   

    plt.title("Net Profit vs. Power in Watts")
    plt.xlabel("Power (W)")
    plt.ylabel("Net Profit ($/hr)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


# Interactive widgets
btc_price_slider = widgets.FloatSlider(value=27500, min=1000, max=60000, step=500, description='BTC Price:')
electricity_rate_slider = widgets.FloatSlider(value=0.030, min=0.001, max=0.50, step=0.001, description='Electricity Rate:', readout_format='.3f')

# Display the interactive plot
widgets.interactive(plot_net_profit, btc_price=btc_price_slider, electricity_rate=electricity_rate_slider)


interactive(children=(FloatSlider(value=27500.0, description='BTC Price:', max=60000.0, min=1000.0, step=500.0…